In [2]:
# We implement a simple version of GPT 2 using the transformer architecture, following attention is all you need paper
# In our case, we build a character level language model, which will be trained on tinyshakespeare dataset
import torch 

In [3]:
# Open data
with open('tinyshakespeare.txt', 'r', encoding = 'utf-8') as f:
    text = f.read() 

In [4]:
print("length of data: ", len(text))

length of data:  1115394


In [5]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [6]:
chars = sorted(list(set(text)))
vocab_size = len(chars) 
print("number of unique characters: ", vocab_size)
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}
print(itos)

number of unique characters:  65
{0: '\n', 1: ' ', 2: '!', 3: '$', 4: '&', 5: "'", 6: ',', 7: '-', 8: '.', 9: '3', 10: ':', 11: ';', 12: '?', 13: 'A', 14: 'B', 15: 'C', 16: 'D', 17: 'E', 18: 'F', 19: 'G', 20: 'H', 21: 'I', 22: 'J', 23: 'K', 24: 'L', 25: 'M', 26: 'N', 27: 'O', 28: 'P', 29: 'Q', 30: 'R', 31: 'S', 32: 'T', 33: 'U', 34: 'V', 35: 'W', 36: 'X', 37: 'Y', 38: 'Z', 39: 'a', 40: 'b', 41: 'c', 42: 'd', 43: 'e', 44: 'f', 45: 'g', 46: 'h', 47: 'i', 48: 'j', 49: 'k', 50: 'l', 51: 'm', 52: 'n', 53: 'o', 54: 'p', 55: 'q', 56: 'r', 57: 's', 58: 't', 59: 'u', 60: 'v', 61: 'w', 62: 'x', 63: 'y', 64: 'z'}


In [7]:
encode = lambda x: [stoi[ch] for ch in x]
decode = lambda x: ''.join([itos[i] for i in x])

print(encode('hello'))
print(decode(encode('hello')))

[46, 43, 50, 50, 53]
hello


In [8]:
data = torch.tensor(encode(text), dtype = torch.long)
print(data.shape, data.dtype)
print(data[:100])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [9]:
# Split in to train anv validation

n = int(0.9 * len(data))
train_data = data[:n]
val_data= data[n:]


In [10]:
block_size = 8
batch_size = 4

In [11]:
torch.manual_seed(1337)

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i: i + block_size] for i in ix])
    y = torch.stack([data[i + 1: i + block_size + 1] for i in ix])
    return x, y

# # visualize a batch
xb, yb = get_batch('train')
print('inputs:')
print(xb.shape, xb.dtype)
print(xb)
print('targets:')
print(yb.shape, yb.dtype)
print(yb)

print("-------")
for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f'When input is {context.tolist()} target is {target.item()}')

inputs:
torch.Size([4, 8]) torch.int64
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8]) torch.int64
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
-------
When input is [24] target is 43
When input is [24, 43] target is 58
When input is [24, 43, 58] target is 5
When input is [24, 43, 58, 5] target is 57
When input is [24, 43, 58, 5, 57] target is 1
When input is [24, 43, 58, 5, 57, 1] target is 46
When input is [24, 43, 58, 5, 57, 1, 46] target is 43
When input is [24, 43, 58, 5, 57, 1, 46, 43] target is 39
When input is [44] target is 53
When input is [44, 53] target is 56
When input is [44, 53, 56] target is 1
When input is [44, 53, 56, 1] target is 58
When input is [44, 53, 56, 1, 58] target is 46
When input is [44, 5

In [12]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)


class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensors. where B is batch size and T is the number of tokens in each sequence (block_size*batch_size)

        logits = self.token_embedding_table(idx)  # (B,T,C)
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            print(logits.shape)
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, _ = self(idx)
            logits = logits[:, -1, :]  # Becomes (B,C)
            probs = F.softmax(logits, dim=-1)  # (B,C)
            idx_next = torch.multinomial(probs, num_samples=1)  # (B,1)
            idx = torch.cat((idx, idx_next), dim=1)  # (B,T+1)
        return idx


m = BigramLanguageModel(vocab_size)
out = m(xb, yb)
print(out[0].shape)
print(out[1])
idx = torch.zeros((1,1), dtype=torch.long)

print(decode(m.generate(idx, 100)[0].tolist()))

torch.Size([4, 8, 65])
torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)

SKIcLT;AcELMoTbvZv C?nq-QE33:CJqkOKH-q;:la!oiywkHjgChzbQ?u!3bLIgwevmyFJGUGp
wnYWmnxKWWev-tDqXErVKLgJ


In [13]:
# Training loop 
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [14]:
batch_size = 32
for steps in range(100000):
    xb, yb = get_batch('train')
    logits, loss = m(xb, yb)
    print(logits.shape)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    if steps % 1000 == 0:
        print(f'step: {steps}, loss: {loss.item()}')

torch.Size([32, 8, 65])
torch.Size([256, 65])
step: 0, loss: 4.692410945892334
torch.Size([32, 8, 65])
torch.Size([256, 65])
torch.Size([32, 8, 65])
torch.Size([256, 65])
torch.Size([32, 8, 65])
torch.Size([256, 65])
torch.Size([32, 8, 65])
torch.Size([256, 65])
torch.Size([32, 8, 65])
torch.Size([256, 65])
torch.Size([32, 8, 65])
torch.Size([256, 65])
torch.Size([32, 8, 65])
torch.Size([256, 65])
torch.Size([32, 8, 65])
torch.Size([256, 65])
torch.Size([32, 8, 65])
torch.Size([256, 65])
torch.Size([32, 8, 65])
torch.Size([256, 65])
torch.Size([32, 8, 65])
torch.Size([256, 65])
torch.Size([32, 8, 65])
torch.Size([256, 65])
torch.Size([32, 8, 65])
torch.Size([256, 65])
torch.Size([32, 8, 65])
torch.Size([256, 65])
torch.Size([32, 8, 65])
torch.Size([256, 65])
torch.Size([32, 8, 65])
torch.Size([256, 65])
torch.Size([32, 8, 65])
torch.Size([256, 65])
torch.Size([32, 8, 65])
torch.Size([256, 65])
torch.Size([32, 8, 65])
torch.Size([256, 65])
torch.Size([32, 8, 65])
torch.Size([256, 65])
t

KeyboardInterrupt: 

In [ ]:
print(decode(m.generate(idx, 500)[0].tolist()))

In [15]:
torch.manual_seed(1337)
B,T,C = 4,8,2 #batch size, time, channels # 
#or batch size, block size, vocab size (vocab size is embedding size)
x = torch.randn(B,T,C)
x.shape


torch.Size([4, 8, 2])

In [16]:
#We want tokens to itnetact with each other, but only "forward in time"
#for now, we calculate the average of the previous tokens plus the current token

In [17]:
xbow = torch.zeros(B,T,C) #xbow for bag of words, because we are not using the order of the tokens
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] 
        xbow[b,t] = torch.mean(xprev,0)

TypeError: mean() received an invalid combination of arguments - got (Tensor, int), but expected one of:
 * (*, torch.dtype dtype)
      didn't match because some of the arguments have invalid types: (!Tensor!, !int!)
 * (tuple of ints dim, bool keepdim, *, torch.dtype dtype)
 * (tuple of names dim, bool keepdim, *, torch.dtype dtype)


In [ ]:
x[0]

NameError: name 'x' is not defined